Préparation des corpus

In [9]:
import sentencepiece as spm
import pandas as pd

# Spécifiez le chemin vers votre fichier CSV
chemin_fichier_csv = 'data/jorf_2023.csv'

# Spécifiez le séparateur (dans ce cas, '|')
separateur = '|'

# Utilisez Pandas pour lire le fichier CSV dans un DataFrame
df = pd.read_csv(chemin_fichier_csv, sep=separateur,names=["ID texte","ID article","Nature","N° article","N° alinéa","Contenu"])

# Affichez le DataFrame
print(df)


                    ID texte            ID article  Nature N° article  \
0       JORFTEXT000048734585                   NaN       0        NaN   
1       JORFTEXT000048734585  JORFVERS000048734585       0        NaN   
2       JORFTEXT000048734585  JORFARTI000048734586       1          1   
3       JORFTEXT000048734585  JORFARTI000048734586       1          1   
4       JORFTEXT000048734585  JORFARTI000048734586       1          1   
...                      ...                   ...     ...        ...   
454296  JORFTEXT000046851183  JORFVERS000046851183       0        NaN   
454297  JORFTEXT000046851183  JORFARTI000046851184       1        NaN   
454298  JORFTEXT000046851186                   NaN       0        NaN   
454299  JORFTEXT000046851186  JORFVERS000046851186       0        NaN   
454300  JORFTEXT000046851186  JORFARTI000046851187       1        NaN   

        N° alinéa                                            Contenu  
0               0                     fr/lr/loi/2023

In [10]:
import re

def extraire_texte_entre_symboles(texte):
    # Utilisez une expression régulière pour extraire le texte entre '«' et '»'
    #match = re.search(r'«(.+)»', texte)
    match = re.search(r'«(.+)', texte) # Beaucoup de phrases n'ont pas le '»' nécessaire
    
    # Vérifiez si une correspondance a été trouvée
    if match:
        texte_extrait = match.group(1)
        return texte_extrait
    else:
        return None

def extraire_texte_en_dehors_des_symboles(texte):
    # Utilisez une expression régulière pour extraire le texte entre '«' et '»'
    match = re.search(r'^[^«]+', texte)
    
    # Vérifiez si une correspondance a été trouvée
    if match:
        texte_extrait = match.group(0)
        return texte_extrait
    else:
        return None

# Exemple d'utilisation
ma_chaine = "Ceci est un exemple «de texte «à extraire» avec des guillemets «pour tester»»."
resultat = extraire_texte_entre_symboles(ma_chaine)

if resultat:
    print("Texte extrait :", resultat)
else:
    print("Aucun texte trouvé entre les symboles '«' et '»'.")
    
resultat = extraire_texte_en_dehors_des_symboles(ma_chaine)

if resultat:
    print("Texte extrait :", resultat)
else:
    print("Aucun texte trouvé en dehors des symboles '«' et '»'.")



Texte extrait : de texte «à extraire» avec des guillemets «pour tester»».
Texte extrait : Ceci est un exemple 


In [11]:
# Création des colonnes ne contenant que les textes relatifs au Droit/non Droit
df['droit'] = df['Contenu'].apply(extraire_texte_entre_symboles)
df['non_droit'] = df['Contenu'].apply(extraire_texte_en_dehors_des_symboles)


In [12]:
# Ecrire l'ensemble du contenu dans un fichier texte
with open("input_data/jorf_2023.txt","w") as f:
    f.writelines("\n".join(df["Contenu"].to_list()))
    
# On extrait que le Droit : entre guillemets
with open("./input_data/jorf_2023_droit.txt","w") as f:
    data = "\n".join(df["droit"].dropna().to_list())
    f.writelines(data)
    
# On extrait que ce qui entoure le Droit
with open("./input_data/jorf_2023_non_droit.txt","w") as f:
    data = "\n".join(df["non_droit"].dropna().to_list())
    f.writelines(data)

    A. Nombre de tokens

In [13]:
for n in (100,1000,10000):
    spm.SentencePieceTrainer.train(input='./input_data/jorf_2023.txt', model_prefix=f'./models/{n}_tokens', vocab_size=n)

In [14]:
# On encode une phrase selon les différents modèles : 100 tokens appara^t clairement insuffisant
for n in [100,1000,10000]:
    sp = spm.SentencePieceProcessor(model_file=f'./models/{n}_tokens.model')
    print(f"Avec {n} tokens :")
    print(sp.EncodeAsPieces("Décret du 29 décembre 2023 portant promotion dans l'ordre national de la Légion d'honneur"))

Avec 100 tokens :
['▁', 'D', 'é', 'c', 're', 't', '▁d', 'u', '▁', '2', '9', '▁d', 'é', 'c', 'e', 'm', 'b', 're', '▁', '2', '0', '2', '3', '▁', 'p', 'o', 'r', 't', 'a', 'n', 't', '▁', 'p', 'r', 'o', 'm', 'o', 't', 'i', 'o', 'n', '▁d', 'a', 'n', 's', '▁', 'l', "'", 'o', 'r', 'd', 're', '▁', 'n', 'a', 't', 'i', 'o', 'n', 'a', 'l', '▁de', '▁', 'l', 'a', '▁', 'L', 'é', 'g', 'i', 'o', 'n', '▁d', "'", 'h', 'o', 'n', 'n', 'e', 'u', 'r']
Avec 1000 tokens :
['▁Décret', '▁du', '▁2', '9', '▁décembre', '▁2023', '▁portant', '▁pro', 'mo', 'tion', '▁dans', '▁l', "'", 'ord', 're', '▁national', '▁de', '▁la', '▁L', 'é', 'g', 'ion', '▁d', "'", 'h', 'onne', 'ur']
Avec 10000 tokens :
['▁Décret', '▁du', '▁29', '▁décembre', '▁2023', '▁portant', '▁promotion', '▁dans', '▁l', "'", 'ordre', '▁national', '▁de', '▁la', '▁Légion', '▁d', "'", 'honneur']


100 tokens sont bien sur insuffisants pour intélligement traiter ce lexique. Avec 1000 tokens on a encore quelques mots qu'on s'attendrait à voir pris dans l'intégralité et pourtant découpés, comme 'Légion' et 'honneur'. Avec 10000 tokens, la tokénisation est presque trop ample, en prenant '29' comme un token en soit.

    B. Droit/non droit

In [15]:
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_droit.txt', model_prefix='./models/droit', vocab_size=1000)
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_non_droit.txt', model_prefix='./models/non_droit', vocab_size=1000)

In [16]:
sp1 = spm.SentencePieceProcessor(model_file=f'./models/droit.model')
sp2 = spm.SentencePieceProcessor(model_file=f'./models/non_droit.model')

# Comparaison de la tokenisation d'une phrase de droit et d'une non de Droit 
s1 = "« L'exonération est attribuée ou retirée dans les conditions prévues aux articles R. 5553-1 et R. 5553-2. »"
s2 = "Arrêté du 26 décembre 2023 relatif aux prix des prestations d'hébergement de certains établissements accueillant des personnes âgées"

for s in [s1,s2]:
    print(s)
    for sp in [sp1,sp2]:
        print(sp.EncodeAsPieces(s))
    print("\n")

« L'exonération est attribuée ou retirée dans les conditions prévues aux articles R. 5553-1 et R. 5553-2. »
['▁«', '▁L', "'", 'ex', 'on', 'é', 'r', 'ation', '▁est', '▁', 'att', 'ri', 'b', 'u', 'ée', '▁ou', '▁re', 'ti', 'ré', 'e', '▁dans', '▁les', '▁conditions', '▁prévues', '▁aux', '▁articles', '▁R', '.', '▁5', '5', '5', '3-1', '▁et', '▁R', '.', '▁5', '5', '5', '3', '-2', '.', '▁»']
['▁', '«', '▁L', "'", 'ex', 'on', 'é', 'r', 'ation', '▁est', '▁', 'at', 't', 'ri', 'b', 'u', 'ée', '▁ou', '▁re', 'ti', 'ré', 'e', '▁dans', '▁les', '▁conditions', '▁prévu', 'es', '▁aux', '▁articles', '▁R', '.', '▁5', '5', '5', '3', '-1', '▁et', '▁R', '.', '▁5', '5', '5', '3', '-', '2', '.', '▁', '»']


Arrêté du 26 décembre 2023 relatif aux prix des prestations d'hébergement de certains établissements accueillant des personnes âgées
['▁A', 'r', 'r', 'ê', 'té', '▁du', '▁2', '6', '▁décembre', '▁2023', '▁relatif', '▁aux', '▁p', 'ri', 'x', '▁des', '▁prestation', 's', '▁d', "'", 'h', 'é', 'b', 'er', 'g', 'ement', 

Les tokenisations Droit/Non Droit ont chacune adopté des tokens adaptés à leur vocabuaire, comme par exemple 'attribué'/'Arrêté'

    C. Avec un plus grand corpus

In [17]:
import pandas as pd
import glob

# Chemin vers le dossier contenant les fichiers CSV
dossier_csv = 'data/'

# Utiliser la fonction glob pour obtenir la liste des fichiers CSV dans le dossier
fichiers_csv = glob.glob(dossier_csv + '*.csv')

# Initialiser une liste pour stocker les DataFrames individuels
dfs = []

# Spécifiez le séparateur (dans ce cas, '|')
separateur = '|'

# Lire chaque fichier CSV et l'ajouter à la liste
for fichier_csv in fichiers_csv:
    df = pd.read_csv(fichier_csv,sep=separateur,names=["ID texte","ID article","Nature","N° article","N° alinéa","Contenu"])
    dfs.append(df)

# Concaténer les DataFrames de la liste en un seul DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame final
print(df_final)


                     ID texte            ID article  Nature N° article  \
0        JORFTEXT000039696471                   NaN       0        NaN   
1        JORFTEXT000039696471  JORFVERS000039696471       0        NaN   
2        JORFTEXT000039696471  JORFARTI000039696480       1          1   
3        JORFTEXT000039696471  JORFARTI000039696482       1          2   
4        JORFTEXT000039696471  JORFARTI000039696482       1          2   
...                       ...                   ...     ...        ...   
2244515  JORFTEXT000046851183  JORFVERS000046851183       0        NaN   
2244516  JORFTEXT000046851183  JORFARTI000046851184       1        NaN   
2244517  JORFTEXT000046851186                   NaN       0        NaN   
2244518  JORFTEXT000046851186  JORFVERS000046851186       0        NaN   
2244519  JORFTEXT000046851186  JORFARTI000046851187       1        NaN   

         N° alinéa                                            Contenu  
0                0                  fr/

In [18]:
# Création des colonnes ne contenant que les textes relatifs au Droit/non Droit
df_final['droit'] = df_final['Contenu'].apply(extraire_texte_entre_symboles)
df_final['non_droit'] = df_final['Contenu'].apply(extraire_texte_en_dehors_des_symboles)


In [19]:
len(df_final["Contenu"].to_list())

2244520

In [20]:
# Ecrire l'ensemble du contenu dans un fichier texte
with open("input_data/jorf_2019_2023.txt","w") as f:
    f.writelines("\n".join(df_final["Contenu"].to_list()))
    
# On extrait que le Droit : entre guillemets
with open("./input_data/jorf_2019_2023_droit.txt","w") as f:
    data = "\n".join(df_final["droit"].dropna().to_list())
    f.writelines(data)
    
# On extrait que ce qui entoure le Droit
with open("./input_data/jorf_2019_2023_non_droit.txt","w") as f:
    data = "\n".join(df_final["non_droit"].dropna().to_list())
    f.writelines(data)

In [21]:
#spm.SentencePieceTrainer.train(input='./input_data/jorf_2019_2023.txt', model_prefix='./models/2019_2023', vocab_size=1000)
spm.SentencePieceTrainer.train(input='./input_data/jorf_2019_2023.txt', model_prefix='./models/2019_2023', vocab_size=1000,shuffle_input_sentence=True)

In [22]:
sp = spm.SentencePieceProcessor(model_file='./models/1000_tokens.model')
print(f"Avec le corpus 2023 :")
print(sp.EncodeAsPieces("Décret du 29 décembre 2023 portant promotion dans l'ordre national de la Légion d'honneur"))

sp = spm.SentencePieceProcessor(model_file='./models/2019_2023.model')
print(f"Avec le corpus 2019-2023 :")
print(sp.EncodeAsPieces("Décret du 29 décembre 2023 portant promotion dans l'ordre national de la Légion d'honneur"))

Avec le corpus 2023 :
['▁Décret', '▁du', '▁2', '9', '▁décembre', '▁2023', '▁portant', '▁pro', 'mo', 'tion', '▁dans', '▁l', "'", 'ord', 're', '▁national', '▁de', '▁la', '▁L', 'é', 'g', 'ion', '▁d', "'", 'h', 'onne', 'ur']
Avec le corpus 2019-2023 :
['▁Décret', '▁du', '▁2', '9', '▁décembre', '▁2023', '▁portant', '▁pro', 'mo', 'tion', '▁dans', '▁l', "'", 'ord', 're', '▁national', '▁de', '▁la', '▁L', 'é', 'g', 'ion', '▁d', "'", 'h', 'onne', 'ur']


In [23]:
sp = spm.SentencePieceProcessor(model_file='./models/1000_tokens.model')
print(f"Avec le corpus 2023 :")
print(sp.EncodeAsPieces("Le dernier alinéa de l'article 3 du décret du 6 octobre 1952 susvisé est remplacé par un alinéa ainsi rédigé : « Les militaires perçoivent, en outre, l'indemnité d'état militaire dans les conditions prévues par le décret n° 59-1193 du 13 octobre 1959 et l'indemnité de garnison des militaires prévue par le décret n° 2023-398 du 24 mai 2023. »"))

sp = spm.SentencePieceProcessor(model_file='./models/2019_2023.model')
print(f"Avec le corpus 2019-2023 :")
print(sp.EncodeAsPieces("Le dernier alinéa de l'article 3 du décret du 6 octobre 1952 susvisé est remplacé par un alinéa ainsi rédigé : « Les militaires perçoivent, en outre, l'indemnité d'état militaire dans les conditions prévues par le décret n° 59-1193 du 13 octobre 1959 et l'indemnité de garnison des militaires prévue par le décret n° 2023-398 du 24 mai 2023. »"))

Avec le corpus 2023 :
['▁Le', '▁dernier', '▁alinéa', '▁de', '▁l', "'", 'article', '▁3', '▁du', '▁décret', '▁du', '▁6', '▁octobre', '▁19', '5', '2', '▁susvisé', '▁est', '▁remplacé', '▁par', '▁un', '▁alinéa', '▁ainsi', '▁rédigé', '▁:', '▁«', '▁Les', '▁militaire', 's', '▁per', 'ç', 'o', 'iv', 'ent', ',', '▁en', '▁ou', 'tre', ',', '▁l', "'", 'indemnité', '▁d', "'", 'éta', 't', '▁militaire', '▁dans', '▁les', '▁conditions', '▁prévues', '▁par', '▁le', '▁décret', '▁n', '°', '▁5', '9', '-', '11', '9', '3', '▁du', '▁', '13', '▁octobre', '▁19', '5', '9', '▁et', '▁l', "'", 'indemnité', '▁de', '▁', 'g', 'ar', 'n', 'is', 'on', '▁des', '▁militaire', 's', '▁prévu', 'e', '▁par', '▁le', '▁décret', '▁n', '°', '▁2023', '-', '3', '9', '8', '▁du', '▁2', '4', '▁mai', '▁2023', '.', '▁»']
Avec le corpus 2019-2023 :
['▁Le', '▁dernier', '▁alinéa', '▁de', '▁l', "'", 'article', '▁3', '▁du', '▁décret', '▁du', '▁6', '▁octobre', '▁19', '5', '2', '▁susvisé', '▁est', '▁remplacé', '▁par', '▁un', '▁alinéa', '▁ainsi', '▁r

In [24]:
sp = spm.SentencePieceProcessor(model_file='./models/1000_tokens.model')
print(f"Avec le corpus 2023 :")
print(sp.EncodeAsPieces("Ceci est une phrase test ayant pour but de voir si l'on parvient à obtenir des différences entre les deux modèles si on utilise un champ lexical peut-être légèrement différeet de ce dont les modèles ont l'habitude"))

sp = spm.SentencePieceProcessor(model_file='./models/2019_2023.model')
print(f"Avec le corpus 2019-2023 :")
print(sp.EncodeAsPieces("Ceci est une phrase test ayant pour but de voir si l'on parvient à obtenir des différences entre les deux modèles si on utilise un champ lexical peut-être légèrement différeet de ce dont les modèles ont l'habitude"))

Avec le corpus 2023 :
['▁C', 'e', 'ci', '▁est', '▁une', '▁phrase', '▁', 't', 'est', '▁ayant', '▁pour', '▁b', 'ut', '▁de', '▁', 'voir', '▁', 's', 'i', '▁l', "'", 'on', '▁par', 'vi', 'ent', '▁à', '▁', 'ob', 'ten', 'ir', '▁des', '▁', 'di', 'ff', 'é', 'r', 'ence', 's', '▁entre', '▁les', '▁deux', '▁', 'mo', 'd', 'è', 'le', 's', '▁', 's', 'i', '▁', 'on', '▁', 'ut', 'ili', 's', 'e', '▁un', '▁champ', '▁l', 'ex', 'ic', 'al', '▁peut', '-', 'ê', 'tre', '▁l', 'é', 'g', 'è', 'r', 'ement', '▁', 'di', 'ff', 'é', 're', 'e', 't', '▁de', '▁ce', '▁dont', '▁les', '▁', 'mo', 'd', 'è', 'le', 's', '▁', 'ont', '▁l', "'", 'h', 'ab', 'it', 'u', 'de']
Avec le corpus 2019-2023 :
['▁C', 'e', 'ci', '▁est', '▁une', '▁phrase', '▁', 'te', 'st', '▁ayant', '▁pour', '▁b', 'ut', '▁de', '▁', 'voir', '▁', 's', 'i', '▁l', "'", 'on', '▁par', 'vi', 'ent', '▁à', '▁', 'ob', 'ten', 'ir', '▁des', '▁d', 'if', 'fér', 'ence', 's', '▁entre', '▁les', '▁deux', '▁', 'mo', 'd', 'è', 'le', 's', '▁', 's', 'i', '▁', 'on', '▁', 'ut', 'il', 'i

Le premier corpus a l'air suffisament conséquent pour qu'il n'y ait pas de différences réelles de tokenistation sur le lexique du corpus. Même en dehors de ce lexique, les différences existent mais sont mineures à 1000 tokens.